In [11]:
from sklearn.metrics import accuracy_score

from labs.helpers import read_dataset
from labs.lab1.classifier import Classifier

In [12]:
data_dir = "../../data/train"
classes = ["field", "road", "town", "water"]
file_pattern = "*{}*.jpg"
standard_shape = (50, 50, 3)

In [13]:
dataset, features = read_dataset(data_dir, classes, file_pattern, standard_shape)
train_df = dataset.sample(frac=0.85, random_state=18)
test_df = dataset.drop(train_df.index)


In [14]:
train_df.head()

,1:1:1,1:1:2,1:1:3,1:2:1,1:2:2,1:2:3,1:3:1,1:3:2,1:3:3,1:4:1,...,50:48:1,50:48:2,50:48:3,50:49:1,50:49:2,50:49:3,50:50:1,50:50:2,50:50:3,class
36,36.0,72.0,72.0,39.0,75.0,75.0,42.0,78.0,78.0,44.0,...,39.0,74.0,76.0,43.0,78.0,80.0,41.0,76.0,78.0,water
26,165.0,187.0,114.0,176.0,202.0,137.0,152.0,186.0,136.0,89.0,...,142.0,185.0,105.0,134.0,172.0,99.0,115.0,153.0,80.0,town
28,228.0,213.0,154.0,228.0,216.0,156.0,230.0,222.0,160.0,231.0,...,250.0,249.0,218.0,251.0,249.0,226.0,251.0,249.0,226.0,town
37,152.0,180.0,121.0,149.0,177.0,118.0,148.0,173.0,115.0,148.0,...,54.0,86.0,85.0,56.0,88.0,87.0,56.0,88.0,87.0,water
16,131.0,150.0,104.0,117.0,139.0,90.0,119.0,144.0,89.0,128.0,...,129.0,169.0,98.0,129.0,166.0,99.0,129.0,166.0,99.0,road


In [15]:
test_df.head()

,1:1:1,1:1:2,1:1:3,1:2:1,1:2:2,1:2:3,1:3:1,1:3:2,1:3:3,1:4:1,...,50:48:1,50:48:2,50:48:3,50:49:1,50:49:2,50:49:3,50:50:1,50:50:2,50:50:3,class
2,187.0,208.0,133.0,184.0,205.0,130.0,181.0,202.0,127.0,180.0,...,145.0,179.0,92.0,143.0,179.0,92.0,143.0,179.0,92.0,field
5,196.0,207.0,131.0,194.0,205.0,129.0,190.0,200.0,127.0,186.0,...,174.0,186.0,114.0,174.0,192.0,116.0,174.0,192.0,116.0,field
8,137.0,171.0,87.0,139.0,173.0,89.0,141.0,175.0,91.0,143.0,...,150.0,184.0,107.0,149.0,183.0,109.0,148.0,182.0,108.0,field
19,204.0,216.0,142.0,197.0,208.0,130.0,190.0,202.0,120.0,187.0,...,178.0,188.0,101.0,166.0,189.0,101.0,165.0,188.0,100.0,road
24,201.0,201.0,129.0,202.0,201.0,134.0,203.0,201.0,140.0,209.0,...,207.0,199.0,162.0,205.0,209.0,150.0,207.0,211.0,152.0,town


In [16]:
model = Classifier()
model.fit(train_df, features, "class")
predicted_classes = model.predict(test_df)
predicted_classes

['field', 'field', 'field', 'road', 'town', 'water', 'water']

In [17]:
real_classes = test_df["class"].values
acc = accuracy_score(test_df["class"], predicted_classes)
print(f"The accuracy score is {acc * 100} %")

The accuracy score is 100.0 %


In [22]:
for i in range(3900, 100, -100):
    local_features_to_use = features[:i]
    model.fit(train_df, local_features_to_use, "class")
    local_predicted_classes = model.predict(test_df)
    local_acc = round(accuracy_score(real_classes, local_predicted_classes), 3)
    print(f"{local_acc * 100}% accuracy score with {len(local_features_to_use)} features")


100.0% accuracy score with 3900 features
100.0% accuracy score with 3800 features
100.0% accuracy score with 3700 features
100.0% accuracy score with 3600 features
100.0% accuracy score with 3500 features
100.0% accuracy score with 3400 features
100.0% accuracy score with 3300 features
100.0% accuracy score with 3200 features
100.0% accuracy score with 3100 features
100.0% accuracy score with 3000 features
100.0% accuracy score with 2900 features
100.0% accuracy score with 2800 features
100.0% accuracy score with 2700 features
100.0% accuracy score with 2600 features
100.0% accuracy score with 2500 features
85.7% accuracy score with 2400 features
85.7% accuracy score with 2300 features
85.7% accuracy score with 2200 features
85.7% accuracy score with 2100 features
85.7% accuracy score with 2000 features
85.7% accuracy score with 1900 features
85.7% accuracy score with 1800 features
85.7% accuracy score with 1700 features
71.39999999999999% accuracy score with 1600 features
71.399999999